# Introduction

> 문서 작성일 : 2025.03.18

GIP에서는 Model 및 prompt에 대한 평가(evaluation) 기능을 지원합니다. 이 문서는 GIP에서 제공하는 평가 기능을 사용하는 기본적인 방법에 대한 소개를 제공합니다.

GIP의 평가 기능은 문서 작성일을 기준으로, 다음과 같은 단계로 구성되어 있습니다.

1. Langfuse credential 연동
2. Experiment 생성: 평가 대상 지정
    - 현재는 Langfuse에 존재하는 데이터 소스에 대한 평가만 지원합니다
        - [Dataset](https://api.reference.langfuse.com/?q=observation#tag/datasets/GET/api/public/v2/datasets/{datasetName})
        - [Trace](https://api.reference.langfuse.com/?q=observation#tag/trace/GET/api/public/traces)
        - [Observation](https://api.reference.langfuse.com/?q=observation#tag/observations/GET/api/public/observations)
3. Evaluator 생성: 평가 방법 지정
    - 현재는 LLM-as-a-Judge 방식의 평가만 지원합니다
    - LLM-as-a-Judge 방식의 평가를 진행하기 위해서는 다음과 같은 정보를 필요로 합니다
        - Judge model with parameters: 평가를 진행하기 위해 사용되는 LLM 모델 및 모델 파라미터 (`top_p`, `temperature`, `max_tokens`)
        - Judge prompt: 평가 방법을 안내하는 프롬프트
        - Metric: 평가 결과가 어떤 형식으로 제공되어야 하는지에 대한 정보
            - 지원하는 Metric 유형
                - Category: `bad` | `neutral` | `good`과 같이 사용자가 임의로 지정한 카테고리
                - Numeric: `0.0` ~ `1.0` 과 같이 범위가 지정된 숫자
                - Bool: `true` / `false`
4. Experiment Run 생성: Experiment 및 evaluator를 이용한 평가 진행
    - 평가 대상(experiment)에 대해서 어떻게 평가할지(evaluator)에 대한 정보의 조합으로 experiment run을 생성함으로서 **실제 평가를 진행합니다**
    - 모델 성능 비교 등의 목적으로 평가를 진행하는 경우, `target_data_converter`를 사용하는 것이 도움이 될 수 있으며, 이에 관해서는 후술하겠습니다
    - Experiment run에는 평가 진행 상황 및 결과가 포함되어 있으며, 평가가 완료된 경우에는 업로드 되어있는 상세 평가 결과를 받아볼 수 있습니다
5. Auto Evaluation Config 생성: 자동 평가 설정
    - 평가 대상(experiment)에 대해서 어떻게 평가할지(evaluator)에 대한 정보의 조합 및 자동 평가 실행 주기(schedule, timezone)을 지정하여 주기적인 **자동 평가를 진행하도록 구성합니다**
    - 자동 평가 실행 주기(schedule)는 cron 표현식을 사용하여 지정할 수 있습니다
    - 지정된 실행 주기에 맞춰서 experiment run을 생성하여 평가를 진행하기 때문에, experiment run 생성을 자동화하는 것으로 보셔도 됩니다.

## 사전 준비

우선 다음의 python 패키지가 설치되어 있어야 합니다.

- `requests`
- `python-dotenv`

환경 변수 파일(`.env`)를 사용하기 위해서는 `.env` 파일에 다음과 같은 내용을 채워주세요.

```plaintext
LANGFUSE_PUBLIC_KEY=pk-******
LANGFUSE_SECRET_KEY=sk-******
LANGFUSE_HOST=https://api.langfuse.com

GIP_CONSOLE_HOST=https://dev-console-api.platform.a15t.com
GIP_CONSOLE_ENTERPRISE_API_KEY=
GIP_WORKSPACE_ID=
```

In [2]:
import os

from dotenv import load_dotenv


load_dotenv() # Optional: 환경 변수 파일(.env)을 불러옵니다


GIP_CONSOLE_HOST = os.getenv('GIP_CONSOLE_HOST', 'https://dev-console-api.platform.a15t.com')
GIP_CONSOLE_ENTERPRISE_API_KEY = os.getenv('GIP_CONSOLE_ENTERPRISE_API_KEY')
GIP_WORKSPACE_ID = os.getenv('GIP_WORKSPACE_ID')

LANGFUSE_HOST = os.getenv('LANGFUSE_HOST', 'https://api.langfuse.com')
LANGFUSE_PUBLIC_KEY = os.getenv('LANGFUSE_PUBLIC_KEY')
LANGFUSE_SECRET_KEY = os.getenv('LANGFUSE_SECRET_KEY')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


### 인증 방법 관련 안내

GIP Console의 API를 호출하는 데에는 두가지 방법이 있습니다.

1. Enterprise API Key 사용
2. GIP Console JWT Token 사용

이 문서는 enterprise API key를 사용하는 것을 기준으로 evaluation API를 활용하는 방법에 대해서 안내하고 있으며, 필요하신 경우 인증 헤더 정보를 GIP Console JWT Token을 사용하도록 변경하시면 됩니다.

#### 1. Enterprise API Key 사용

In [5]:
import requests


response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

#### 2. GIP Console JWT Token 사용

In [11]:
import requests


# GIP Console 로그인을 통해서 JWT Token을 발급받는 과정
response = requests.post(
    url='https://cognito-idp.ap-northeast-2.amazonaws.com',
    headers={
        'Content-Type': 'application/x-amz-json-1.1',
        'X-Amz-Target': 'AWSCognitoIdentityProviderService.InitiateAuth',
    },
    json={
        'AuthParameters': {
            # GIP Console 로그인 시에 사용하는 계정 정보 기재
            'USERNAME': 'keep-secret',
            'PASSWORD': 'keep-secret',
        },
        'AuthFlow': 'USER_PASSWORD_AUTH',
        'ClientId': '4dm722bemutkce2500cck1bbk4', # dev 환경용 client ID -> prod 환경의 GIP Console API 호출시에는 별도 문의 필요
    },
)

response.raise_for_status()

auth_response = response.json()
id_token = auth_response['AuthenticationResult']['IdToken']

# GIP Console 로그인을 통해서 얻은 JWT Token을 사용해서 GIP Console API 호출
response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'Authorization': f'Bearer {id_token}'}
)

response.raise_for_status()

## 1. Langfuse credential 연동

현재 GIP에서는 Langfuse에 존재하는 데이터 소스에 대한 평가만 우선적으로 지원하기 때문에, Langfuse API key 연동이 필요합니다.

Langfuse의 API key는 각 프로젝트 별로 설정되며, 다음의 문서를 통해서 API key 생성 / 조회 방법을 확인하실 수 있습니다.

[Where are my Langfuse API keys?](https://langfuse.com/faq/all/where-are-langfuse-api-keys)

Langfuse API key를 확인한 이후에는 다음의 과정을 거쳐서 GIP에 연동하실 수 있습니다.

In [6]:
import requests


response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/integrated-services/credentials',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'service_name': 'LANGFUSE',
        'alias': 'gip-eval-intro-demo', # GIP <-> Langfuse API Key 연동시의 별칭 (Langfuse project name 등으로 설정하시면 편합니다)
        'base_url': LANGFUSE_HOST,
        'public_key': LANGFUSE_PUBLIC_KEY,
        'private_key': LANGFUSE_SECRET_KEY,
    },
)

response.raise_for_status()

integrated_service_credential = response.json()
integrated_service_credential_id = integrated_service_credential['id']

print('Integrated service credential ID: ', integrated_service_credential_id)

Integrated service credential ID:  ccd1ea20-5d1c-47e8-9a4b-a36ced7cb856


## 2. Experiment 생성

Experiment는 평가 대상에 대한 정보를 갖고 있습니다. 현재는 Langfuse에 존재하는 데이터 소스에 대한 평가만 지원하고 있으며, 다음의 데이터 소스에 대한 평가를 지원합니다.

- [Dataset](https://api.reference.langfuse.com/?q=observation#tag/datasets/GET/api/public/v2/datasets/{datasetName})
- [Trace](https://api.reference.langfuse.com/?q=observation#tag/trace/GET/api/public/traces)
- [Observation](https://api.reference.langfuse.com/?q=observation#tag/observations/GET/api/public/observations)

각각의 데이터 소스에 대해서 experiment를 생성하는 방법은 다음과 같습니다.

### 2.1 Dataset

In [10]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Langfuse dataset > `JSONL_samples-plan_suggestion_dataset_v2`',
        'description': 'GIP evaluation introduction demo - dataset 평가 대상 지정', # Optional
        'type': 'langfuse_dataset',
        'config': {
            'integrated_service_credential_id': integrated_service_credential_id,
            'dataset_name': 'JSONL_samples-plan_suggestion_dataset_v2', # 평가 대상이 되는 Langfuse dataset 이름 (WARN: 현재 GIP에서는 dataset 이름에 공백이 없는 경우만 지원합니다)
            'include_archived_items': False, # Dataset 내의 아카이브된 데이터 포함 여부
        },
    },
)

response.raise_for_status()

dataset_experiment = response.json()
dataset_experiment_id = dataset_experiment['id']

print('Experiment ID (dataset): ', dataset_experiment_id)

Experiment ID (dataset):  0195a8cd-2752-7e32-a822-5a92ad7d81dd


### 2.2 Trace

In [11]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Langfuse trace',
        'description': 'GIP evaluation introduction demo - trace 평가 대상 지정', # Optional
        'type': 'langfuse_trace',
        'config': {
            'integrated_service_credential_id': integrated_service_credential_id,
            'sampling_rate': 1.0, # 0.0 ~ 1.0 사이의 값으로 설정
            'max_trace_count': 10, # 추출할 trace의 최대 개수
            'trace_filter_request': { # Trace 필터링 조건 - Langfuse API Reference 참조: https://api.reference.langfuse.com/#tag/trace/GET/api/public/traces
                'name': None,
                'from_timestamp': None,
                'to_timestamp': 1742293735000,
                'tags': None,
                'version': None,
                'release': None,
                'relative_time_range': None,
            }
        },
    },
)

response.raise_for_status()

trace_experiment = response.json()
trace_experiment_id = trace_experiment['id']

print('Experiment ID (trace): ', trace_experiment_id)

Experiment ID (trace):  0195a8cd-5603-70c2-86ef-412d8bb21451


### 2.3 Observation

In [ ]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Langfuse observation',
        'description': 'GIP evaluation introduction demo - observation 평가 대상 지정', # Optional
        'type': 'langfuse_observation',
        'config': {
            'integrated_service_credential_id': integrated_service_credential_id,
            'sampling_rate': 1.0, # 0.0 ~ 1.0 사이의 값으로 설정
            'max_observation_count': 10, # 추출할 observation의 최대 개수
            'observation_filter_request': { # Observation 필터링 조건 - Langfuse API Reference 참조: https://api.reference.langfuse.com/#tag/observations/GET/api/public/observations
                'name': None,
                'from_start_time': None,
                'to_start_time': 1742293735000,
                'version': None,
                'relative_time_range': None,
            }
        },
    },
)

response.raise_for_status()

observation_experiment = response.json()
observation_experiment_id = observation_experiment['id']

print('Experiment ID (observation): ', observation_experiment_id)

experiment의 `type` 값이 `langfuse_observation` 혹은 `langfuse_trace`인 경우 다음의 필드를 활용하여 고정 시간 범위 내의 데이터를 조회할 수 있습니다.

- `observation_filter_request.from_start_time`
- `observation_filter_request.to_start_time`
- `trace_filter_request.from_timestamp`
- `trace_filter_request.to_timestamp`

이는 experiment run 혹은 auto evaluation config를 통한 평가를 진행할때, 평가 진행 시점을 기준으로 한 상대적인 데이터를 조회하는 데에 제약사항이 되며, GIP에서는 이러한 제약사항을 해결하기 위해 `relative_time_range`를 지원합니다.

`relative_time_range` 필드를 활용하면 다음의 필드 값을 무시하게 되며, `relative_time_range`가 최우선시 되어 적용됩니다.

- `observation_filter_request.from_start_time`
- `observation_filter_request.to_start_time`
- `trace_filter_request.from_timestamp`
- `trace_filter_request.to_timestamp`

`relative_time_range` 필드 값은 [ISO-8601의 duration 형식](https://en.wikipedia.org/wiki/ISO_8601#Durations)을 따르며, 최근 12시간 동안의 데이터를 조회하고 싶은 경우 `PT12H`로 기재해서 사용하시면 됩니다.

### Trace 적용 예시

In [ ]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Langfuse trace + `relative_time_range`',
        'description': 'GIP evaluation introduction demo - trace 평가 대상 지정 w/ relative time range', # Optional
        'type': 'langfuse_trace',
        'config': {
            'integrated_service_credential_id': integrated_service_credential_id,
            'sampling_rate': 1.0,
            'max_trace_count': 10,
            'trace_filter_request': {
                'name': None,
                'from_timestamp': None,
                'to_timestamp': None,
                'tags': None,
                'version': None,
                'release': None,
                'relative_time_range': 'PT12H', # NOTE: 최근 12시간 동안의 데이터
            }
        },
    },
)

response.raise_for_status()

### Observation 적용 예시

In [ ]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Langfuse observation + `relative_time_range`',
        'description': 'GIP evaluation introduction demo - observation 평가 대상 지정 w/ relative time range', # Optional
        'type': 'langfuse_observation',
        'config': {
            'integrated_service_credential_id': integrated_service_credential_id,
            'sampling_rate': 1.0,
            'max_observation_count': 10,
            'observation_filter_request': {
                'name': None,
                'from_start_time': None,
                'to_start_time': None,
                'version': None,
                'relative_time_range': 'PT24H', # NOTE: 최근 24시간 동안의 데이터
            }
        },
    },
)

response.raise_for_status()

## 3. Evaluator 생성

Evaluator는 평가 방법에 대한 정보를 갖고 있습니다. 현재는 LLM-as-a-Judge 방식의 평가만 지원하고 있으며, LLM-as-a-Judge 방식의 평가를 진행하기 위해서는 다음과 같은 정보를 필요로 합니다.

- Judge model with parameters: 평가를 진행하기 위해 사용되는 LLM 모델 및 모델 파라미터 (`top_p`, `temperature`, `max_tokens`)
- Judge prompt: 평가 방법을 안내하는 프롬프트
- Metric: 평가 결과가 어떤 형식으로 제공되어야 하는지에 대한 정보

Judge prompt의 예시는 다음과 같습니다:

<details>
<summary>Judge prompt example</summary>
<p>

```markdown
You are an impartial AI judge. Your task is to evaluate whether the given **output** is correct based on the provided **input**.

## Evaluation Criteria
**Accuracy**: Does the output provide correct?
**Explanation:** Does the output provide an explanation or justification for its statements?

## Instructions for Evaluation
1. Carefully review the input and corresponding output.
2. Assess the output based on the criteria above.
3. Provide a detailed explanation for each criterion, noting strengths and weaknesses.
4. Assign an overall accuracy rating: `correct` (highly accurate) or `incorrect` (highly inaccurate).

## Answer Format

**Evaluation Rationale:**
<rationale>
Provide a detailed explanation for each criterion.
</rationale>

**Overall Accuracy Rating:**
<rating>
Provide an overall accuracy rating here based on the evaluation rationale. Follow the evaluation instructions.
</rating>

## Example Evaluation

**Input:** `[ { 'role': 'user', 'content': 'Is a fox a member of the dog family?' } ]`


**Output:** `{ 'role': 'assistant', 'content': 'Yes' }`

**Evaluation Rationale:**
<rationale>
- **Accuracy**: Correct. A typical fox is a member of the dog family.
- **Explanation:** The output does not provide an explanation, but the answer is concise and accurate.
</rationale>

**Overall Accuracy Rating:**
<rating>
correct
</rating>

Let’s think step by step.
```

</p>
</details>

현재 지원되는 metric의 유형은 다음과 같습니다.

- Category: `bad` | `neutral` | `good`과 같이 사용자가 임의로 지정한 카테고리
- Numeric: `0.0` ~ `1.0` 과 같이 범위가 지정된 숫자
- Bool: `true` / `false`

각각의 metric 유형에 대해서 evaluator를 생성하는 방법은 다음과 같습니다.

### 3.1 Category Metric


In [16]:
judge_prompt = '''
You are an impartial AI judge. Your task is to evaluate whether the given **output** is correct based on the provided **input**.

## Evaluation Criteria
**Accuracy**: Does the output provide correct?
**Explanation:** Does the output provide an explanation or justification for its statements?

## Instructions for Evaluation
1. Carefully review the input and corresponding output.
2. Assess the output based on the criteria above.
3. Provide a detailed explanation for each criterion, noting strengths and weaknesses.
4. Assign an overall accuracy rating: `correct` (highly accurate) or `incorrect` (highly inaccurate).

## Answer Format

**Evaluation Rationale:**
<rationale>
Provide a detailed explanation for each criterion.
</rationale>

**Overall Accuracy Rating:**
<rating>
Provide an overall accuracy rating here based on the evaluation rationale. Follow the evaluation instructions.
</rating>

## Example Evaluation

**Input:**
```json
[ { 'role': 'user', 'content': 'Is a fox a member of the dog family?' } ]
```

**Output:**
```json
{ 'role': 'assistant', 'content': 'Yes' }
```

**Evaluation Rationale:**
<rationale>
- **Accuracy**: Correct. A typical fox is a member of the dog family.
- **Explanation:** The output does not provide an explanation, but the answer is concise and accurate.
</rationale>

**Overall Accuracy Rating:**
<rating>
correct
</rating>

Let’s think step by step.
'''.strip()

response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/evaluators',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Category Metric Evaluator',
        'metric': {
            'type': 'category',
            'config': [
                {'index': 0, 'name':  'bad'},
                {'index': 1, 'name':  'neutral'},
                {'index': 2, 'name':  'good'},
            ],
        },
        'type': 'llm_judge',
        'evaluation_config': {
            'judge_model_seq_id': 19,
            'judge_model_public_id': 'openai/gpt-4',
            'judge_model_parameters': {
                'top_p': 0.5,
                'temperature': 0.5,
                'max_tokens': 50,
            },
            'judge_prompt': judge_prompt,
        },
    },
)

response.raise_for_status()

category_metric_evaluator = response.json()
category_metric_evaluator_id = category_metric_evaluator['id']

print('Evaluator ID (category metric): ', category_metric_evaluator_id)

Evaluator ID (category metric):  0195a8f7-224e-7163-8feb-12c4ca1ae5ad


### 3.2 Numeric Metric

In [17]:
judge_prompt = '''
You are an impartial AI judge. Your task is to evaluate whether the given **output** is correct based on the provided **input**.

## Evaluation Criteria
**Factual Accuracy:** Are the facts, figures, and information presented in the output accurate and verifiable?
**Logical Consistency:** Does the output follow a logical sequence and make sense in the context of the input? Are there any logical fallacies or inconsistencies?
**Adherence to Requirements:** Does the output meet the specific requirements or criteria outlined in the input? Are all necessary conditions and constraints satisfied?
**Precision:** Is the information in the output precise and specific? Are there any vague or ambiguous statements that could lead to misinterpretation?
**Error-Free:** Is the output free from typographical, grammatical, or computational errors that could affect its correctness?

## Instructions for Evaluation
1. Carefully review the input and corresponding output.
2. Assess the output based on the criteria above.
3. Provide a detailed explanation for each criterion, noting strengths and weaknesses.
4. Assign an overall accuracy rating between `1.0` (highly accurate) and `0.0` (highly inaccurate).

## Answer Format

**Evaluation Rationale:**
<rationale>
Provide a detailed explanation for each criterion.
</rationale>

**Overall Accuracy Rating:**
<rating>
Provide an overall accuracy rating here based on the evaluation rationale. Follow the evaluation instructions.
</rating>

## Example Evaluation

**Input:**
How many legs does a cat have?

**Output:**
2

**Evaluation Rationale:**
<rationale>
- **Factual Accuracy:** Incorrect. A typical cat has 4 legs, not 2.
- **Logical Consistency:** The output does not logically follow from the input.
- **Adherence to Requirements:** The output fails to provide the correct number of legs.
- **Precision:** While specific, the number '2' is incorrect.
- **Error-Free:** No grammatical errors, but the factual inaccuracy affects correctness.
</rationale>

**Overall Accuracy Rating:**
<rating>
0.0
</rating>

Let’s think step by step.
'''.strip()

response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/evaluators',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Numeric Metric Evaluator',
        'metric': {
            'type': 'numeric',
            'config': {
                'min': {
                    'value': 0.0,
                },
                'max': {
                    'value': 1.0,
                },
            },
        },
        'type': 'llm_judge',
        'evaluation_config': {
            'judge_model_seq_id': 19,
            'judge_model_public_id': 'openai/gpt-4',
            'judge_model_parameters': {
                'top_p': 0.5,
                'temperature': 0.5,
                'max_tokens': 50,
            },
            'judge_prompt': judge_prompt,
        },
    },
)

response.raise_for_status()

numeric_metric_evaluator = response.json()
numeric_metric_evaluator_id = numeric_metric_evaluator['id']

print('Evaluator ID (numeric metric): ', numeric_metric_evaluator_id)

Evaluator ID (numeric metric):  0195a8f7-b924-7b90-9484-477ea0863b82


### 3.3 Bool Metric

In [18]:
judge_prompt = '''
You are an impartial AI judge. Your task is to evaluate whether the given **output** is correct based on the provided **input**.

## Evaluation Criteria
**Factual Accuracy:** Are the facts, figures, and information presented in the output accurate and verifiable?
**Logical Consistency:** Does the output follow a logical sequence and make sense in the context of the input? Are there any logical fallacies or inconsistencies?
**Adherence to Requirements:** Does the output meet the specific requirements or criteria outlined in the input? Are all necessary conditions and constraints satisfied?
**Precision:** Is the information in the output precise and specific? Are there any vague or ambiguous statements that could lead to misinterpretation?
**Error-Free:** Is the output free from typographical, grammatical, or computational errors that could affect its correctness?

## Instructions for Evaluation
1. Carefully review the input and corresponding output.
2. Assess the output based on the criteria above.
3. Provide a detailed explanation for each criterion, noting strengths and weaknesses.
4. Assign an overall accuracy rating: `true` (highly accurate) or `false` (highly inaccurate).

## Answer Format

**Evaluation Rationale:**
<rationale>
Provide a detailed explanation for each criterion.
</rationale>

**Overall Accuracy Rating:**
<rating>
Provide an overall accuracy rating here based on the evaluation rationale. Follow the evaluation instructions.
</rating>

## Example Evaluation

**Input:**
How many legs does a cat have?

**Output:**
2

**Evaluation Rationale:**
<rationale>
- **Factual Accuracy:** Incorrect. A typical cat has 4 legs, not 2.
- **Logical Consistency:** The output does not logically follow from the input.
- **Adherence to Requirements:** The output fails to provide the correct number of legs.
- **Precision:** While specific, the number '2' is incorrect.
- **Error-Free:** No grammatical errors, but the factual inaccuracy affects correctness.
</rationale>

**Overall Accuracy Rating:**
<rating>
false
</rating>

Let’s think step by step.
'''.strip()

response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/evaluators',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'name': 'Numeric Metric Evaluator',
        'metric': {
            'type': 'bool',
        },
        'type': 'llm_judge',
        'evaluation_config': {
            'judge_model_seq_id': 19,
            'judge_model_public_id': 'openai/gpt-4',
            'judge_model_parameters': {
                'top_p': 0.5,
                'temperature': 0.5,
                'max_tokens': 50,
            },
            'judge_prompt': judge_prompt,
        },
    },
)

response.raise_for_status()

bool_metric_evaluator = response.json()
bool_metric_evaluator_id = bool_metric_evaluator['id']

print('Evaluator ID (bool metric): ', bool_metric_evaluator_id)

Evaluator ID (bool metric):  0195a8f9-13a6-7851-8f81-a2e80509871a


## 4. Experiment Run 생성

Experiment와 evaluator의 조합으로 experiment run을 생성하면 평가가 진행됩니다. Experiment run은 평가 진행 상황 및 결과에 대한 정보를 포함합니다.

기본적인 형태의 experiment run을 생성하는 과정은 다음과 같습니다.

In [19]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': category_metric_evaluator_id,
        'display_name': 'Langfuse dataset + category metric experiment run',
        'target_data_converter': None,
        'metadata': None,
    },
)

response.raise_for_status()

basic_experiment_run = response.json()
basic_experiment_run_id = basic_experiment_run['id']

print('Experiment Run ID (basic): ', basic_experiment_run_id)

Experiment Run ID (basic):  0195a909-0e0b-75f1-99d9-78c9082a9210


평가 진행 과정에서 다소 시간이 걸릴 수 있기 때문에, experiment run의 상태를 확인한 후에 상세 결과를 확인해보겠습니다.

In [36]:
response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/runs/{basic_experiment_run_id}',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

basic_experiment_run = response.json()

print('Basic experiment run: ')
print('\tstatus: ', basic_experiment_run['status'])
print('\taggregated_result:\n\t', basic_experiment_run['aggregated_result'])
print('\tresult_path: ', basic_experiment_run['result_path'])

Basic experiment run: 
	status:  DONE
	aggregated_result:
	 {'error': None, 'total_usage': {'prompt_tokens': 751295, 'completion_tokens': 26534, 'total_tokens': 777829, 'web_grounding_requests': 0, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'text_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': 0}}}
	result_path:  workspaces/2d6d2b89-8454-40a1-8d85-0aaba543f47e/experiment_runs/0195a909-0e0b-75f1-99d9-78c9082a9210.csv


experiment run의 상태가 `DONE`이 되면, 상세 결과를 확인할 수 있습니다. 상세 결과는 다음과 같이 확인 가능합니다.

In [42]:
import csv
import io

response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/runs/{basic_experiment_run_id}/download',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

csv_content = response.content.decode('utf-8')
csv_reader = csv.DictReader(io.StringIO(csv_content))

for index, row in enumerate(csv_reader):
    if index > 5: # NOTE: 상세 결과의 일부만 확인
        break

    print('Row #', index)
    print('\texperiment input:\n\t\t', row['input_data_from_experiment'])
    print('\texperiment run input:\n\t\t', row['input_data_from_experiment_run'])
    print('\texperiment output:\n\t\t', row['output_data_from_experiment'])
    print('\texperiment run output:\n\t\t', row['output_data_from_experiment_run'])
    print('\tpassed:', row['passed'])
    print('\terror_message:', row['error_message'] or 'x') # NOTE: `passed` 값이 `false`인 경우에 어떤 문제가 있었는지에 대해 나타냄
    print('\tscore:', row['score'])
    print('\trationale:\n\t\t', row['rationale'])

이렇게 기본적인 experiment run은 experiment에 지정된 데이터 소스로부터 input/output prompt를 가져온 후, evaluator에 지정된 judge prompt를 통한 평가를 진행하게 됩니다. 이는 input/output prompt에 대한 평가를 지원하는 것입니다.

다만, model 별로 어떤 output이 나오는지에 대해서 평가하고 싶은 상황에 대해서는 기본적인 experiment run을 생성하는 것으로 부족합니다. 이런 경우에는 `target_data_converter`를 사용하여 평가 대상 데이터를 변환하는 과정을 추가할 수 있습니다.

`target_data_converter`를 사용하게 되면, exeperiment에 지정된 데이터 소스로부터 input prompt를 가져온 후에 `target_data_converter`를 거쳐서 한번 더 input/output prompt가 가공됩니다. 평가 대상이 `target_data_converter`를 거친 가공된 input/output prompt가 되는 것입니다.

자세한 사항은 다음의 예시를 통해 확인해보겠습니다.

In [43]:
# 1. model: `anthropic/claude-3-sonnet`
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': category_metric_evaluator_id,
        'display_name': 'Langfuse dataset + category metric experiment run + target data converter (anthropic/claude-3-sonnet-20240229)',
        'target_data_converter': {
            'model_seq_id': 6,
            'model_public_id': 'anthropic/claude-3-sonnet-20240229',
            'model_parameters': {
                'top_p': 0.5,
                'temperature': 0.6,
                'max_tokens': 50,
            },
            'input_prompt_trajectory_template': {
                'messages': [
                    {
                        'role': 'system',
                        'content': 'You are an AI assistant that answers questions',
                    },
                    {
                        'role': 'user',
                        'content': '{item.input}',
                    },
                ],
            },
        },
    },
)

response.raise_for_status()

sonnet_experiment_run = response.json()
sonnet_experiment_run_id = sonnet_experiment_run['id']

# 2. model: `anthropic/claude-3-opus`
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': category_metric_evaluator_id,
        'display_name': 'Langfuse dataset + category metric experiment run + target data converter (anthropic/claude-3-opus-20240229)',
        'target_data_converter': {
            'model_seq_id': 5,
            'model_public_id': 'anthropic/claude-3-opus-20240229',
            'model_parameters': {
                'top_p': 0.5,
                'temperature': 0.6,
                'max_tokens': 50,
            },
            'input_prompt_trajectory_template': {
                'messages': [
                    {
                        'role': 'system',
                        'content': 'You are an AI assistant that answers questions',
                    },
                    {
                        'role': 'user',
                        'content': '{item.input}',
                    },
                ],
            },
        },
    },
)

response.raise_for_status()

opus_experiment_run = response.json()
opus_experiment_run_id = opus_experiment_run['id']

print('Experiment Run ID (anthropic/claude-3-sonnet-20240229): ', sonnet_experiment_run_id)
print('Experiment Run ID (anthropic/claude-3-opus-20240229): ', opus_experiment_run_id)

Experiment Run ID (anthropic/claude-3-sonnet-20240229):  0195a945-7c09-7782-8e00-a82c9a6f496b
Experiment Run ID (anthropic/claude-3-opus-20240229):  0195a945-7c9d-7d31-8276-2e59380b109d


위의 예시는 같은 Langfuse dataset의 데이터에 대해서 `anthropic/claude-3-sonnet`과 `anthropic/claude-3-opus` 모델의 성능을 비교하기 위한 experiment run을 생성한 것입니다. 각각의 결과를 통해서 모델간의 성능 차이를 확인할 수 있습니다.

또한, experiment run 생성 요청에 나온 것처럼 `input_prompt_trajectory_template`를 통해서 각 모델에 대한 input prompt를 지정할 수 있습니다. system prompt를 변경하는 상황에 대한 비교가 필요한 경우에 이 점을 활용하는 것이 유용할 수 있습니다.

추가로, `input_prompt_trajectory_template`의 값을 자세히 보면 `role: user`인 부분에 `{item.input}`이라는 템플릿이 사용된 것을 확인하실 수 있습니다. 이는 experiment에 지정된 데이터를 가져오는 과정에서 어떤 데이터를 가져올 수 있을지 지정하는 것입니다.

예를 들어 experiment의 `type`이 `langfuse_dataset`인 경우, [DatasetItem](https://api.reference.langfuse.com/#tag/datasetitems/GET/api/public/dataset-items) 객체를 가져오게 됩니다. 해당 데이터에는 `input` 및 `expectedOutput`과 같은 속성 값들이 포함되어 있으며, 이를 활용하여 `input_prompt_trajectory_template`를 구성하는 것이 가능합니다. 만약에 `DatasetItem`의 `expectedOutput`을 사용하고 싶은 경우에는 `{item.input}`대신에 `{item.expectedOutput}` 표현을 사용하면 됩니다.

experiment의 `type` 값에 따른 접근 가능한 객체는 다음과 같습니다.

- `langfuse_dataset`: [DatasetItem](https://api.reference.langfuse.com/#tag/datasetitems/GET/api/public/dataset-items)
- `langfuse_trace`: [TraceWithDetails](https://api.reference.langfuse.com/#tag/trace/GET/api/public/traces)
- `langfuse_observation`: [ObservationsViews](https://api.reference.langfuse.com/#tag/observations/GET/api/public/observations)


## Auto Evaluation

GIP에서는 지정된 일정에 맞춰서 주기적으로 평가를 진행할 수 있도록 auto evaluation 기능을 지원합니다. auto evaluation을 위해서는 다음의 과정을 거쳐야 합니다.

1. Langfuse credential 연동
2. Experiment 생성: 평가 대상 지정
3. Evaluator 생성: 평가 방법 지정
4. Auto evaluation config 생성
    - 평가 대상(experiment)에 대해서 어떻게 평가할지(evaluator)에 대한 정보의 조합으로 정해진 일정에 맞춰 experiment run을 생성하도록 지원합니다
    - 지정된 일정에 따라 주기적으로 평가가 진행됩니다

전반적인 과정은 앞서 살펴본 experiment run 생성과 동일하며, experiment run 생성을 auto evaluation config로 어느정도 대체하는 것으로 보는 것이 이해하시는 데에 도움이 될 수 있습니다.

auto evaluation config를 생성하는 예시는 다음과 같습니다.

In [ ]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{dataset_experiment_id}/auto-evaluations',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': category_metric_evaluator_id,
        'enabled': True,
        'name': 'Langfuse dataset + category metric auto evaluation (auto)',
        'schedule': '0 23 * * *', # cron expression: 매일 23시에 평가를 진행
        'timezone': 'Asia/Seoul',  # default: `Etc/UTC`
        'target_data_converter': None,
        'metadata': None,
    },
)

response.raise_for_status()

basic_auto_evaluation_config = response.json()
basic_auto_evaluation_config_id = basic_auto_evaluation_config['id']

print('Auto Evaluation Config ID (basic): ', basic_auto_evaluation_config_id)

위의 예시를 통해서 auto evaluation config를 생성한 것을 확인하실 수 있습니다. auto evaluation config를 생성하게 되면 지정된 일정(`schedule`, `timezone`)에 따라 experiment run이 생성되며, 자동으로 주기적인 평가가 진행됩니다.

또한, experiment run 생성과 마찬가지로 `target_data_converter`를 지원하기 때문에 평가 대상 데이터를 변환하는 과정을 추가할 수 있습니다. 이를 통해서 평가 대상 데이터를 가공하여 자동으로 평가를 진행하는 것도 가능합니다.